<font size="5">Interactive Wildfire Map for Uttarakhand, India</font>

This project has been inspired by the paper, Hotspot and trend analysis of forest fires and its relation to climatic factors in the western Himalayas, linked below. The paper studies wildfire patterns in the Western Himalayan region over the course of 2001 - 2020 with the conclusion that Uttarakhand experiences the most number of wildfires around the year with a spike in the month of May and a drop after Monsoon. 

Through this project I have attempted to reproduce the findings of the paper utilizing the wildfire data from the year 2024 and the boundary of the state of Uttarakhand, India. The output html file features a dropdown to select month so you can see for yourself how the conclusion of the paper holds.

<a href= "https://link.springer.com/article/10.1007/s11069-022-05530-5" target= "_blank">Kumar, Sunil, and Amit Kumar. “Hotspot and trend analysis of forest fires and its relation to climatic factors in the western Himalayas.” Natural hazards (Dordrecht, Netherlands) vol. 114,3 (2022): 3529-3544. doi:10.1007/s11069-022-05530-5</a>

In [1]:
# Importing necessary packages
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import calendar
from IPython.display import display

In [2]:
# Reading shapefiles into geodataframes
fires_jan_may = gpd.read_file("F:/spatial_analysis/project_1/DL_FIRE_M-C61_519846/fire_archive_M-C61_519846.shp")
fires_jun_aug = gpd.read_file("F:/spatial_analysis/project_1/DL_FIRE_M-C61_519846/fire_nrt_M-C61_519846.shp")
uttarakhand = gpd.read_file("F:/spatial_analysis/project_1/uttarakhand.shp")

# Printing the shapes of the two fire geodataframes
print("Data points from the month of January 2024 up till May is: ", 
      fires_jan_may.shape[0], 
      '\n',
      "Data points from June to August is: ", 
      fires_jun_aug.shape[0])

Data points from the month of January 2024 up till May is:  10419 
 Data points from June to August is:  2191


In [3]:
# Combining the data from January to August to create one geodataframe 
fires = gpd.GeoDataFrame(pd.concat([fires_jan_may, fires_jun_aug], ignore_index= True))

# Confirming the shape of the geodataframe
print(fires.shape[0])

12610


In [4]:
# Extracting data points that intersect the boundary of Uttarakhand and creating a new geodataframe
uttarakhand_fires = gpd.overlay(df1= fires, df2= uttarakhand, how= "intersection", keep_geom_type= True)

# Printing all the columns in the uttarakhand_fires geodataframe
print(uttarakhand_fires.columns, '\n')

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'fid', 'GID_1', 'GID_0',
       'COUNTRY', 'NAME_1', 'VARNAME_1', 'NL_NAME_1', 'TYPE_1', 'ENGTYPE_1',
       'CC_1', 'HASC_1', 'ISO_1', 'geometry'],
      dtype='object') 



In [5]:
# Removing dataframes that will not be necessary for our analysis
uttarakhand_fires = uttarakhand_fires.drop(columns= ['TYPE', 'fid', 'GID_1', 'GID_0',
       'COUNTRY', 'NAME_1', 'VARNAME_1', 'NL_NAME_1', 'TYPE_1', 'ENGTYPE_1',
       'CC_1', 'HASC_1', 'ISO_1'])

# Converting column names to lowercase for ease and uniformity
uttarakhand_fires.columns = uttarakhand_fires.columns.str.lower()
print(uttarakhand_fires.columns)

Index(['latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_date',
       'acq_time', 'satellite', 'instrument', 'confidence', 'version',
       'bright_t31', 'frp', 'daynight', 'geometry'],
      dtype='object')


In [6]:
# Creating a month column by extracting month from the date column 
uttarakhand_fires['month'] = uttarakhand_fires['acq_date'].dt.month

# Creating a month name column for better user readability
uttarakhand_fires['month_name'] = uttarakhand_fires['acq_date'].dt.month_name
uttarakhand_fires['acq_date'] = uttarakhand_fires['acq_date'].astype(str)

# Creating a geodataframe containing the number of wildfire occurences per month
wildfire_count_per_month = uttarakhand_fires.groupby(['month']).size().reset_index(name='fire_count_per_month')
print(wildfire_count_per_month)

   month  fire_count_per_month
0      1                   289
1      2                   112
2      3                    72
3      4                   867
4      5                   939
5      6                   443
6      7                     4
7      8                     6


In [7]:
# Merging the wildfire count per month geodataframe to the uttarakhand fires 
uttarakhand_fires = uttarakhand_fires.merge(wildfire_count_per_month, on = ['month'], how = 'left')
print(uttarakhand_fires.head(2))

   latitude  longitude  brightness  scan  track    acq_date acq_time  \
0   31.0805    78.2029       300.8   1.5    1.2  2024-01-01     0522   
1   29.3042    79.9958       300.9   1.0    1.0  2024-01-01     0821   

  satellite instrument  confidence version  bright_t31   frp daynight  \
0     Terra      MODIS          18   61.03       284.3  11.5        D   
1      Aqua      MODIS          37   61.03       290.0   4.0        D   

                  geometry  month  \
0  POINT (78.2029 31.0805)      1   
1  POINT (79.9958 29.3042)      1   

                                          month_name  fire_count_per_month  
0  <bound method PandasDelegate._add_delegate_acc...                   289  
1  <bound method PandasDelegate._add_delegate_acc...                   289  


In [8]:
# Creating an interactive map 
def generate_interactive_map():
    
    # Creating traces for each month
    data = []
    # List for fire counts each month
    fire_counts = []

    for month in range(1, 9):  # Months from January to August
        fires_in_month = uttarakhand_fires[uttarakhand_fires['month'] == month]
        fire_count = fires_in_month.shape[0]  # Calculate fire count for each month
        fire_counts.append(fire_count)  # Storing fire count
        
        trace = go.Scattermapbox(
            lat=fires_in_month.geometry.y,
            lon=fires_in_month.geometry.x,
            mode='markers',
            marker=go.scattermapbox.Marker(size=7, color='firebrick'),
            text=fires_in_month['acq_date'],
            hoverinfo='text',
            name=calendar.month_name[month],
            visible=(month == 1)
        )
        data.append(trace)

    # Setting up the map layout
    layout = go.Layout(
        mapbox=dict(
            style="carto-positron",
            center={"lat": uttarakhand.geometry.centroid.y.mean(), "lon": uttarakhand.geometry.centroid.x.mean()},
            zoom=7
        ),
        title=f"Wildfires in Uttarakhand in 2024 - Monthly total: {fire_counts[0]}",  # Initially set to January's count
        margin={"r": 0, "t": 100, "l": 0, "b": 50},  # Adjusted bottom margin for footer space
        updatemenus=[
            dict(
                active=0,
                buttons=[
                    dict(label=calendar.month_name[i],
                         method="update",
                         args=[{"visible": [i == month for month in range(1, 9)]},
                                {"title": f"Wildfires in Uttarakhand in 2024 - Monthly total : {fire_counts[i-1]}"}])  # Update title
                    for i in range(1, 9)
                ],
                direction="down",
                showactive=True,
            )
        ],
         annotations=[
    
            dict(
                text="Created by: Purvanii Pragya",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.5, y=-0.02, 
                xanchor="center", yanchor="top",
                font=dict(size=12, color='#333333'),
                align="center",
                width=400
            ),
           
            dict(
                text="Data Source: <a href='https://firms.modaps.eosdis.nasa.gov/' target='_blank' style='color:blue;'>NASA FIRMS</a>",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.5, y=-0.06, 
                xanchor="center", yanchor="top",
                font=dict(size=12, color='#333333'),
                align="center",
                width=400
            )
        ]
    )


    # Creating the figure
    fig = go.Figure(data=data, layout=layout)
    return fig


# Generating the interactive map
interactive_map = generate_interactive_map()

# Saving the map in an html file
interactive_map.write_html("interactive_wildfire_map_with_dropdown.html")

C:\Users\windows\AppData\Local\Temp\ipykernel_5340\1431092720.py:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center={"lat": uttarakhand.geometry.centroid.y.mean(), "lon": uttarakhand.geometry.centroid.x.mean()},
